In [1]:
import pandas as pd
import numpy as np

Load review dataset

In [2]:
products = pd.read_csv('/content/amazon_baby_subset.csv')

listing the name of the first 10 products in the dataset.

In [3]:
products['name'][:10]

0    Stop Pacifier Sucking without tears with Thumb...
1      Nature's Lullabies Second Year Sticker Calendar
2      Nature's Lullabies Second Year Sticker Calendar
3                          Lamaze Peekaboo, I Love You
4    SoftPlay Peek-A-Boo Where's Elmo A Children's ...
5                            Our Baby Girl Memory Book
6    Hunnt&reg; Falling Flowers and Birds Kids Nurs...
7    Blessed By Pope Benedict XVI Divine Mercy Full...
8    Cloth Diaper Pins Stainless Steel Traditional ...
9    Cloth Diaper Pins Stainless Steel Traditional ...
Name: name, dtype: object

counting the number of positive and negative reviews.

In [4]:
print ((products['sentiment'] == 1).sum())
print ((products['sentiment'] == -1).sum())
print ((products['sentiment']).count())

26579
26493
53072


Apply text cleaning on the review data

In [6]:
import json
with open('/content/important_words.json') as important_words_file:    
    important_words = json.load(important_words_file)
print(important_words[:3])

['baby', 'one', 'great']


data transformations:

fill n/a values in the review column with empty strings.

Remove punctuation.

Compute word counts (only for important_words).

In [8]:
products = products.fillna({'review':''})  # fill in N/A's in the review column

def remove_punctuation(text):
    import string
    return text.translate(string.punctuation) 

products['review_clean'] = products['review'].apply(remove_punctuation)
products.head(3)

,name,review,rating,sentiment,review_clean
0,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,1,All of my kids have cried non-stop when I trie...
1,Nature's Lullabies Second Year Sticker Calendar,We wanted to get something to keep track of ou...,5,1,We wanted to get something to keep track of ou...
2,Nature's Lullabies Second Year Sticker Calendar,My daughter had her 1st baby over a year ago. ...,5,1,My daughter had her 1st baby over a year ago. ...


Compute a count for the number of times the word occurs in the review

In [9]:
for word in important_words:
    products[word] = products['review_clean'].apply(lambda s : s.split().count(word))

In [10]:
products.head(1)

,name,review,rating,sentiment,review_clean,baby,one,great,love,use,would,like,easy,little,seat,old,well,get,also,really,son,time,bought,product,good,daughter,much,loves,stroller,put,months,car,still,back,used,recommend,first,even,perfect,nice,...,looks,second,piece,box,pretty,trying,difficult,together,though,give,started,anything,last,company,come,returned,maybe,took,broke,makes,stay,instead,idea,head,said,less,went,working,high,unit,seems,picture,completely,wish,buying,babies,won,tub,almost,either
0,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,1,All of my kids have cried non-stop when I trie...,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Compute the number of product reviews that contain the word perfect.

In [11]:
products['contains_perfect'] = products['perfect'] >=1
print (products['contains_perfect'].sum())

2303


Convert data frame to multi-dimensional array

In [19]:
def get_numpy_data(dataframe, features, label):
    dataframe['constant'] = 1
    features = ['constant'] + features
    features_frame = dataframe[features]
    feature_matrix = features_frame.to_numpy()
    label_sarray = dataframe[label]
    label_array = label_sarray.to_numpy()
    return(feature_matrix, label_array)

extract two arrays feature_matrix and sentiment

In [20]:
feature_matrix, sentiment = get_numpy_data(products, important_words, 'sentiment')

In [21]:
print(feature_matrix.shape)

(53072, 194)


Estimating conditional probability with link function

In [23]:
'''
feature_matrix: N * D
coefficients: D * 1
predictions: N * 1
produces probablistic estimate for P(y_i = +1 | x_i, w).
estimate ranges between 0 and 1.
'''

def predict_probability(feature_matrix, coefficients):
    # Take dot product of feature_matrix and coefficients  
    # YOUR CODE HERE
    score = np.dot(feature_matrix, coefficients) # N * 1
    
    # Compute P(y_i = +1 | x_i, w) using the link function
    # YOUR CODE HERE
    predictions = 1.0/(1+np.exp(-score))
    
    # return predictions
    return predictions

Compute derivative of log likelihood with respect to a single coefficient

In [25]:
"""
errors: N * 1
feature: N * 1
derivative: 1 
"""
def feature_derivative(errors, feature):     
    # Compute the dot product of errors and feature
    derivative = np.dot(np.transpose(errors), feature)
    # Return the derivative
    return derivative

In [26]:
def compute_log_likelihood(feature_matrix, sentiment, coefficients):
    indicator = (sentiment==+1)
    scores = np.dot(feature_matrix, coefficients)
    # scores.shape (53072L, 1L)
    # indicator.shape (53072L,)
    lp = np.sum((np.transpose(np.array([indicator]))-1)*scores - np.log(1. + np.exp(-scores)))
    return lp

Taking gradient steps

In [35]:
# coefficients: D * 1
from math import sqrt
def logistic_regression(feature_matrix, sentiment, initial_coefficients, step_size, max_iter):
    coefficients = np.array(initial_coefficients) # make sure it's a numpy array
    # lplist = []
    for itr in range(max_iter):
        # Predict P(y_i = +1|x_1,w) using your predict_probability() function
        # YOUR CODE HERE
        predictions = predict_probability(feature_matrix, coefficients)

        # Compute indicator value for (y_i = +1)
        indicator = (sentiment==+1)

        # Compute the errors as indicator - predictions
        errors = np.transpose(np.array([indicator])) - predictions

        for j in range(len(coefficients)): # loop over each coefficient
            # Recall that feature_matrix[:,j] is the feature column associated with coefficients[j]
            # compute the derivative for coefficients[j]. Save it in a variable called derivative
            # YOUR CODE HERE
            derivative = feature_derivative(errors, feature_matrix[:,j])

            # add the step size times the derivative to the current coefficient
            # YOUR CODE HERE
            coefficients[j] += step_size*derivative

        # Checking whether log likelihood is increasing
        if itr <= 15 or (itr <= 100 and itr % 10 == 0) or (itr <= 1000 and itr % 100 == 0) \
        or (itr <= 10000 and itr % 1000 == 0) or itr % 10000 == 0:
            # lplist.append(compute_log_likelihood(feature_matrix, sentiment, coefficients))
            lp = compute_log_likelihood(feature_matrix, sentiment, coefficients)
            print ('iteration %*d: log likelihood of observed labels = %.8f' % \
                (int(np.ceil(np.log10(max_iter))), itr, lp))
    """
    import matplotlib.pyplot as plt
    x= [i for i in range(len(lplist))]
    plt.plot(x,lplist,'ro')
    plt.show()
    """
    return coefficients

run the logistic regression solver

In [36]:
initial_coefficients = np.zeros((194,1))
step_size = 1e-7
max_iter = 301

In [39]:
coefficients = logistic_regression(feature_matrix, sentiment, initial_coefficients, step_size, max_iter)

iteration   0: log likelihood of observed labels = -36782.24149905
iteration   1: log likelihood of observed labels = -36777.77993493
iteration   2: log likelihood of observed labels = -36773.32246359
iteration   3: log likelihood of observed labels = -36768.86907436
iteration   4: log likelihood of observed labels = -36764.41975666
iteration   5: log likelihood of observed labels = -36759.97449997
iteration   6: log likelihood of observed labels = -36755.53329383
iteration   7: log likelihood of observed labels = -36751.09612785
iteration   8: log likelihood of observed labels = -36746.66299174
iteration   9: log likelihood of observed labels = -36742.23387522
iteration  10: log likelihood of observed labels = -36737.80876812
iteration  11: log likelihood of observed labels = -36733.38766031
iteration  12: log likelihood of observed labels = -36728.97054176
iteration  13: log likelihood of observed labels = -36724.55740245
iteration  14: log likelihood of observed labels = -36720.1482

Compute class predictions

In [41]:
"""
feature_matrix: N * D
coefficients: D * 1
predictions: N * 1
"""
predictions = predict_probability(feature_matrix, coefficients)
NumPositive = (predictions > 0.5).sum()
print (NumPositive)

score = np.dot(feature_matrix, coefficients) # N * 1
print ((score > 0).sum())

24714
24714


Measuring accuracy

In [43]:
print (0 in products['sentiment'])

True


In [44]:
print (-1 in products['sentiment'])

False


In [45]:
print (np.transpose(predictions.flatten()).shape)
print ((products['sentiment']).shape)

(53072,)
(53072,)


In [46]:
print ((np.transpose(predictions.flatten()))[:5])

[0.50977952 0.49260815 0.50289701 0.50055279 0.52735675]


In [47]:
correct_num = np.sum((np.transpose(predictions.flatten())> 0.5) == np.array(products['sentiment']>0))
total_num = len(products['sentiment'])
print ("correct_num: {}, total_num: {}".format(correct_num, total_num))
accuracy = correct_num * 1./ total_num
print (accuracy)

correct_num: 39117, total_num: 53072
0.737055321073259


In [48]:
np.transpose(predictions.flatten())> 0.5

array([ True, False,  True, ..., False, False, False])

In [49]:
np.array(products['sentiment']>0)

array([ True,  True,  True, ..., False, False, False])

In [50]:
correct_num = np.sum((np.transpose(score.flatten())> 0) == np.array(products['sentiment']>0))
total_num = len(products['sentiment'])
print ("correct_num: {}, total_num: {}".format(correct_num, total_num))
accuracy = correct_num * 1./ total_num
print (accuracy)

correct_num: 39117, total_num: 53072
0.737055321073259


Compute the "most positive words"

In [51]:
coefficients = list(coefficients[1:]) # exclude intercept
word_coefficient_tuples = [(word, coefficient) for word, coefficient in zip(important_words, coefficients)]
word_coefficient_tuples = sorted(word_coefficient_tuples, key=lambda x:x[1], reverse=True)

Compute the 10 most positive words

In [52]:
word_coefficient_tuples[:10]

[('love', array([0.06516289])),
 ('easy', array([0.06202955])),
 ('great', array([0.05068325])),
 ('little', array([0.04496421])),
 ('loves', array([0.04487736])),
 ('perfect', array([0.02268598])),
 ('well', array([0.02069822])),
 ('fits', array([0.01708208])),
 ('nice', array([0.01654066])),
 ('daughter', array([0.01645556]))]

Ten "most negative" words

In [53]:
word_coefficient_tuples[-10:]

[('monitor', array([-0.02068181])),
 ('thought', array([-0.02075643])),
 ('work', array([-0.02094667])),
 ('money', array([-0.02235262])),
 ('waste', array([-0.02315535])),
 ('return', array([-0.02457066])),
 ('get', array([-0.029324])),
 ('even', array([-0.03021414])),
 ('product', array([-0.03617626])),
 ('would', array([-0.05387976]))]